<a href="https://colab.research.google.com/github/kobemawu/www/blob/master/LDA_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# nltkの文書群にgensimのLDAを適用してみる

## 準備

必要ライブラリ(全部pipで入ります)
 * nltk
 * gensim
 * pyLDAvis

In [ ]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis

In [1]:
#nltk使ったことない人は、pip install してから対話環境等で以下のdataset等をダウンロードしてください
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## データロード・前処理

In [2]:
#dataset読み込み
from nltk.corpus import reuters as corpus

### 今回はこういう文書(をBOW化したもの)を用います

In [ ]:
#必要に応じて以下のコードを実行してください．
!unzip /root/nltk_data/corpora/reuters.zip -d /root/nltk_data/corpora


In [3]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:300]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")

ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears  
among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They  
told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And  
lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long -  
run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will  
impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to  
stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at  
10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports 

In [4]:
#全document数
len(corpus.fileids())

10788

In [5]:

#前からk個のdocumentのみで学習する場合
#k=1000
#docs=[corpus.words(fileid) for fileid in corpus.fileids()[:k]]

#全documentで学習する場合
docs=[corpus.words(fileid) for fileid in corpus.fileids()]

print(docs[:5])
print("num of docs:", len(docs))

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', ...], ['CHINA', 'DAILY', 'SAYS', 'VERMIN', 'EAT', '7', '-', ...], ['JAPAN', 'TO', 'REVISE', 'LONG', '-', 'TERM', ...], ['THAI', 'TRADE', 'DEFICIT', 'WIDENS', 'IN', 'FIRST', ...], ['INDONESIA', 'SEES', 'CPO', 'PRICE', 'RISING', ...]]
num of docs: 10788


## 前処理 

In [6]:
#ストップワードリストの作成

#1 nltkのストップワードリスト
en_stop = nltk.corpus.stopwords.words('english')

# 一度LDAしてみる等して，適宜ノイズになってそうな記号等を見つけて，ストップワードリストに新たに加える
#【発展】記号や数字は正規表現で消してみましょう
en_stop= ["``", "/", ";", "--", ":", ")", "(", '"', '&', "'", '-', "?", ">", "<"]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price"]          \
         +en_stop

In [7]:
#前処理関数の作成

from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

def preprocess_word(word, stopwordset):
    
    #1.make words lower  example: Python =>python
    word=word.lower()
    
    #2.remove "," and "."
    if word in [",","."]:
        return None
    
    #3.remove stopword  example: the => (None) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset: #lemmatizeしたものがstopwordである可能性がある
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

In [8]:
#before
print(docs[0][:25]) 

#after
print(preprocess_documents(docs)[0][:25])

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears']
['asian', 'exporter', 'fear', 'damage', '.-', 'japan', 'rift', 'mounting', 'trade', 'friction', 'japan', 'raise', 'fear', 'among', 'many', 'asia', 'exporting', 'nation', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage']


## LDA準備

In [9]:
import gensim
from gensim import corpora

In [10]:
#documentを，gensim LDAが読み込めるデータ構造にする

#辞書の作成
dictionary = corpora.Dictionary(preprocess_documents(docs))
#コーパスの作成
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

In [11]:
#Dictionary:gensimにおける辞書クラス
#token2id属性には単語と辞書IDとの対応が格納される

print(dictionary.token2id)

{'),': 0, ',"': 1, '.,': 2, '.,"': 3, '.-': 4, '15': 5, '17': 6, '1985': 7, '30': 8, '300': 9, '53': 10, '95': 11, '>.': 12, 'able': 13, 'accounting': 14, 'action': 15, 'advantage': 16, 'allege': 17, 'allow': 18, 'also': 19, 'american': 20, 'among': 21, 'analyst': 22, 'april': 23, 'asia': 24, 'asian': 25, 'ask': 26, 'association': 27, 'australia': 28, 'australian': 29, 'avow': 30, 'await': 31, 'aware': 32, 'barrier': 33, 'beef': 34, 'beyond': 35, 'big': 36, 'block': 37, 'boost': 38, 'broker': 39, 'budget': 40, 'business': 41, 'businessmen': 42, 'button': 43, 'call': 44, 'canberra': 45, 'capel': 46, 'capital': 47, 'centre': 48, 'chairman': 49, 'chief': 50, 'co': 51, 'coal': 52, 'commercial': 53, 'complete': 54, 'concern': 55, 'conflict': 56, 'continue': 57, 'correspondent': 58, 'cost': 59, 'could': 60, 'country': 61, 'curb': 62, 'cut': 63, 'damage': 64, 'day': 65, 'defuse': 66, 'democratic': 67, 'deputy': 68, 'despite': 69, 'deterioration': 70, 'diplomatic': 71, 'director': 72, 'disadva

In [12]:
#corpusにはdocumentごとに単語の(ID、出現回数)のリストが得られる

print(corpus_[0][:10]) #文章での出現順でなく辞書IDの若い順なことに注意

[(0, 1), (1, 6), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]


In [13]:
#before
print([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]])

#after
print(dictionary.doc2bow([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]]))

#これを全文書の全文に適用したのがcorpus_

['asian', 'exporters', 'fear', 'damage', 'from', 'u', '.', 's', '.-', 'japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'u', '.', 's', '.', 'and', 'japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.']
[(4, 1), (21, 1), (24, 1), (25, 1), (42, 1), (60, 1), (64, 2), (77, 1), (90, 1), (93, 1), (94, 1), (101, 1), (121, 1), (125, 2), (147, 1), (162, 1), (203, 1), (214, 1), (215, 1), (261, 1)]


## LDA学習

In [14]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=20,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

## パラメータの確認

In [24]:
#(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 

topics = ldamodel.print_topics(num_words=3)
for topic in topics:
    print(topic)

(0, '0.023*"january" + 0.022*"february" + 0.020*"1986"')
(1, '0.048*"bank" + 0.026*"dollar" + 0.020*"dealer"')
(2, '0.019*"bill" + 0.017*"would" + 0.016*"house"')
(3, '0.023*"oil" + 0.017*"gulf" + 0.016*"opec"')
(4, '0.008*"would" + 0.007*"group" + 0.007*"standard"')
(5, '0.014*"50" + 0.012*"000" + 0.012*"southern"')
(6, '0.077*"stg" + 0.037*"bank" + 0.026*"k"')
(7, '0.017*"offer" + 0.016*"inc" + 0.013*"stock"')
(8, '0.024*"stock" + 0.021*"split" + 0.020*"dividend"')
(9, '0.029*"canadian" + 0.020*"canada" + 0.015*"ltd"')
(10, '0.024*"trade" + 0.012*"market" + 0.012*"japan"')
(11, '0.029*"oil" + 0.012*"energy" + 0.011*"gas"')
(12, '0.081*"000" + 0.055*"loss" + 0.030*"profit"')
(13, '0.022*"franc" + 0.020*"oil" + 0.017*"barrels"')
(14, '0.021*"quarter" + 0.020*"1986" + 0.016*"first"')
(15, '0.019*"sugar" + 0.016*"corn" + 0.014*"000"')
(16, '0.014*"shipment" + 0.012*"loan" + 0.010*"champion"')
(17, '0.039*"div" + 0.037*"april" + 0.037*"record"')
(18, '0.014*"would" + 0.010*","" + 0.007*"m

In [27]:
#[(当該documentにおけるトピックIDとそのprobability　)]　 ※　のうち、minimum_probabilityの値を超えるもの

for n,item in enumerate(corpus_[0:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(7, 0.024801597), (10, 0.8807745), (11, 0.01710977), (18, 0.073469125)]
document ID 1:[(0, 0.16623574), (4, 0.044760104), (10, 0.12810737), (13, 0.24149157), (15, 0.39248782)]
document ID 2:[(0, 0.17132726), (10, 0.30143368), (11, 0.4725894), (19, 0.040255044)]
document ID 3:[(0, 0.6313528), (2, 0.013675144), (11, 0.012908458), (14, 0.32489902)]
document ID 4:[(0, 0.49079916), (6, 0.047235962), (7, 0.013060624), (16, 0.027962072), (18, 0.405426)]
document ID 5:[(3, 0.06361162), (4, 0.10643532), (10, 0.24666278), (15, 0.35596395), (17, 0.10681201), (18, 0.108134136)]
document ID 6:[(0, 0.2800606), (4, 0.11292647), (10, 0.59982073)]
document ID 7:[(5, 0.35401443), (8, 0.17270687), (15, 0.42012224)]
document ID 8:[(7, 0.08170708), (19, 0.8928143)]
document ID 9:[(1, 0.063812986), (2, 0.010470782), (4, 0.038508467), (7, 0.14625537), (10, 0.44951674), (13, 0.013050511), (14, 0.10985065), (18, 0.15670656)]


In [17]:
#documentのcategory
categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

In [35]:
n=5

#n番目のdocumentのトピック分布
print(ldamodel.get_document_topics(corpus_[n]))

#n番目のdocumentのcategory
print(categories[n])

#n番目のdocumentの生の文章
#print(" ".join(docs[n]))

[(3, 0.063622996), (4, 0.106477104), (10, 0.2465981), (15, 0.35596293), (17, 0.10681047), (18, 0.10814822)]
['ship']


## 可視化

In [19]:
#import pyLDAvis.gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

In [20]:
#全documentを学習に用いた場合結構時間がかかる(20min~)
#gensimではK個のトピックに0~K-1のidが割り振られていたのに対し，pyLDAvisでは1~Kのidが割り振られていることに注意

lda_display = pyLDAvis.gensim_models.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [25]:
#上で可視化したモデルをgoogle drive上にsaveできる

pyLDAvis.save_html(lda_display,'vis.html')

In [26]:
!ls

sample_data  vis.html
